In [1]:
!git clone https://github.com/pruizf/disco.git

Cloning into 'disco'...
Updating files:  13% (2412/17252)
Updating files:  14% (2416/17252)
Updating files:  15% (2588/17252)
Updating files:  16% (2761/17252)
Updating files:  17% (2933/17252)
Updating files:  18% (3106/17252)
Updating files:  19% (3278/17252)
Updating files:  20% (3451/17252)
Updating files:  21% (3623/17252)
Updating files:  22% (3796/17252)
Updating files:  23% (3968/17252)
Updating files:  24% (4141/17252)
Updating files:  25% (4313/17252)
Updating files:  26% (4486/17252)
Updating files:  27% (4659/17252)
Updating files:  28% (4831/17252)
Updating files:  29% (5004/17252)
Updating files:  30% (5176/17252)
Updating files:  31% (5349/17252)
Updating files:  31% (5373/17252)
Updating files:  32% (5521/17252)
Updating files:  33% (5694/17252)
Updating files:  34% (5866/17252)
Updating files:  35% (6039/17252)
Updating files:  36% (6211/17252)
Updating files:  37% (6384/17252)
Updating files:  38% (6556/17252)
Updating files:  39% (6729/17252)
Updating files:  40% (69

In [1]:
!git clone https://github.com/v-laurent/VRD-image-text-generator.git

Cloning into 'VRD-image-text-generator'...


In [1]:
%cd "C:\Users\Vivek Kumar\OneDrive\Documents\ml_models_vscode\VRD-image-text-generator"

[WinError 2] The system cannot find the file specified: 'C:\\Users\\Vivek Kumar\\OneDrive\\Documents\\ml_models_vscode\\VRD-image-text-generator'
d:\ml_models_vscode


C:\Users\Vivek Kumar\AppData\Roaming\Python\Python312\site-packages\IPython\core\magics\osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})


In [14]:
!pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ------------ --------------------------- 0.8/2.5 MB 5.6 MB/s eta 0:00:01
   ---------------------------------------- 2.5/2.5 MB 7.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/198.5 MB ? eta -:--:--
    --------------------------------------- 3.1/198.5 MB 15.4 MB/s eta 0:00:13
   - -------------------------------------- 7.6/198.5 MB 18.1 MB/s eta 0:00:11
   -- ------------------------------------- 10.2/198.5 MB 16.8 MB/s eta 0:00:12
   --- ------------------------------------ 14.9/198.5 MB 18.1 MB/s eta 0:00:11
   --- ------------------------------------ 17.0/198.5 MB 17.1 MB/s eta 0:00:11
   ---- ----------------------------------- 21.2/198.5 MB 17.0 MB/s eta 0:00:11
   ---- ----------------------------------- 24.4/198.5 MB 16.8 MB/s eta 0:00:11
   ----- ---------------------------------- 26.7/198.5 MB 16.3 MB/s eta 

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 3.0.1 requires requests>=2.32.2, but you have requests 2.31.0 which is incompatible.
torchaudio 2.5.1+cu121 requires torch==2.5.1+cu121, but you have torch 2.2.2 which is incompatible.

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:

import os
import subprocess
from pathlib import Path

In [72]:
import pandas as pd
df = pd.DataFrame(columns = ['path' , 'text'])

In [2]:

# Step 2: Generate images from extracted text
import random
from multiprocessing import Manager
def generate_images(txt_path , mode , i , shared_list):
    font_sizes= [ "25" ,"30", "35", "40", "45"]
    boolean_values = ["True", "False"]

    # Randomly select values for each argument
    
    with open(txt_path, "r", encoding="utf-8") as file:
        j = 0
        for line in file:
            font_size = random.choice(font_sizes)
            bars = "False"
            add_random_text = "False"
            add_boxes = "False"
            apply_data_augmentation = random.choice(boolean_values)
            j += 1
            line = line.strip()
            if line != '':
                if mode == "word":  # Generate image for each word
                    words = line.split()
                    for word in words:
                        output_path =  f"/teamspace/studios/this_studio/GEN_IMAGES/{word}.png"
                        subprocess.run([
                            "python", "generate_image.py",
                            "--text", word,
                            "--font_size", "40",
                            "--font_path", "",
                            "--bars=False",
                            "--add_random_text=False",
                            "--add_boxes=False",
                            "--apply_data_augmentation=False",
                            "--output_path", output_path
                        ])
                elif mode == "line":  # Generate image for each line
                    output_path = f"/teamspace/studios/this_studio/GEN_IMAGES/image_{i}_{j}.png"
                    command = [
                        "python", "generate_image.py",
                        "--text", line,
                        "--font_size", font_size,
                        "--font_path", "", 
                        "--bars", bars,
                        "--add_random_text",add_random_text,
                        "--add_boxes", add_boxes,
                        "--apply_data_augmentation", apply_data_augmentation,
                        "--output_path", output_path
                    ]
                    result = subprocess.run(command, capture_output=True, text=True)
                    if result.returncode != 0:
                        print(f"Error processing {line}: {result.stderr}")
                shared_list.append((os.path.basename(output_path), line))
    return f'Image saved'
                


In [3]:
import os
max_workers = max(1, os.cpu_count() // 2)  # Use half the available CPU cores


In [4]:
max_workers

16

In [3]:
from concurrent.futures import ProcessPoolExecutor , as_completed
from tqdm import tqdm
import pandas as pd

def process_all_files_parallel(txt_folder, mode="word", max_workers=10):
    txt_files = [os.path.join(txt_folder, f) for f in os.listdir(txt_folder) if f.endswith(".txt")]

    with Manager() as manager:
        shared_list = manager.list()  # Shared list across processes
        
        with ProcessPoolExecutor(max_workers=max_workers) as executor:
            futures = {executor.submit(generate_images, txt_file, mode, i, shared_list): (txt_file, i) 
                       for i, txt_file in enumerate(txt_files)}

            with tqdm(total=len(txt_files), desc="Processing TXT Files", unit="file") as pbar:
                for future in as_completed(futures):
                    txt_file, i = futures[future]  # Retrieve the txt file and index
                    try:
                        result = future.result()  # Ensure errors in processing are raised
                        #print(result)  # Print success messages
                    except Exception as e:
                        print(f"Error processing {txt_file} (index {i}): {e}")
                    pbar.update(1)

        # Convert the shared list to a DataFrame after parallel execution
        df = pd.DataFrame(list(shared_list), columns=["path", "text"])
        df.to_csv('GEN_IMAGES_DF.csv')
        print("\nFinal DataFrame:\n", df.head())
        return df

In [5]:
txt_folder = r'/teamspace/studios/this_studio/txt_sonnet' # Update with the correct path
df = process_all_files_parallel(txt_folder, mode="line", max_workers=16)

Processing TXT Files:   0%|          | 0/1088 [00:00<?, ?file/s]

Processing TXT Files: 100%|██████████| 1088/1088 [53:09<00:00,  2.93s/file] 



Final DataFrame:
              path                                     text
0   image_5_1.png     Nunca ofendí la fe con la esperanza;
1   image_0_1.png      Valencia insigne, patria venturosa,
2  image_12_1.png       Clarín, que rosicleres troglodita,
3  image_11_1.png  Tiemble la tierra, y con furor horrendo
4   image_3_1.png   De tu muerte que fue un breve suspiro,


In [6]:
len(df)

15368